In [15]:
#Primero, a partr del top_50_post_per_user.py, para poder entrenar necesitamos agrupar los  posts por usuario y calcular un embedding por cada usuario que codifique su body de posts

from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

model = SentenceTransformer("all-MiniLM-L6-v2")

# Cargar datos
df = pd.read_csv("C:\\Users\\sam20\\OneDrive\\Documentos\\IA\\CuartoIA\\Tecnologias_del_lenguaje\\practica4\\material\\top50_posts_per_user.csv")

In [17]:
# Calcular embedding por post
embeddings = model.encode(df["post"].tolist(), show_progress_bar=True)
df["embedding"] = list(embeddings)


user_embeddings = (
    df.groupby("username")["embedding"]
      .apply(lambda x: np.mean(np.vstack(x), axis=0)) 
      .reset_index()
)

Batches: 100%|██████████| 2179/2179 [10:45<00:00,  3.37it/s]


In [47]:
authors_train = pd.read_csv("C:\\Users\\sam20\\OneDrive\\Documentos\\IA\\CuartoIA\\Tecnologias_del_lenguaje\\practica4\\material\\authors_train.csv")

# Lista de columnas de personalidad
cols_big5 = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]

# Imputar la media por columna
for col in cols_big5:
    mean_value = authors_train[col].mean()
    authors_train[col].fillna(mean_value, inplace=True)

# Comprobar que ya no quedan NaN
print(authors_train[cols_big5].isna().sum())


data_para_entrenar = user_embeddings.merge(authors_train, on="username")






openness             0
conscientiousness    0
extraversion         0
agreeableness        0
neuroticism          0
dtype: int64


C:\Users\sam20\AppData\Local\Temp\ipykernel_5736\2603811931.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  authors_train[col].fillna(mean_value, inplace=True)


In [49]:
data_para_entrenar.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1288 entries, 0 to 1287
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   username           1288 non-null   object 
 1   embedding          1288 non-null   object 
 2   agreeableness      1288 non-null   float64
 3   openness           1288 non-null   float64
 4   conscientiousness  1288 non-null   float64
 5   extraversion       1288 non-null   float64
 6   neuroticism        1288 non-null   float64
dtypes: float64(5), object(2)
memory usage: 70.6+ KB


In [53]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = np.vstack(data_para_entrenar["embedding"])
y = data_para_entrenar[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=300, random_state=42))
model_rf.fit(X_train, y_train)

y_pred = model_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print("MSE por rasgo:", mse)

MSE por rasgo: [805.0187049  876.26315678 879.85333197 923.70795868 945.51521175]
